In [ ]:
# Data preparation for training Lightning Pose models
# 1. Convert Jarvis labeled data to Lightning Pose labeled data 
# 2. Format videos
# 3. Get context frames
# 

In [ ]:
# Install packages later (Github/hand_tracking/utils/convert_videos)  
import os
import shutil
import pandas as pd
from utils.convert_videos import get_videos  # type: ignore
from utils.jarvis2lp import J2LP_mulview, J2LP_sigview, get_labeledframes, get_contextframes  # type: ignore



## Convert Jarvis labeled data to Lightning Pose labeled data

In [ ]:
# Single-view project
# session_name: YYYY-MM-DD
# trial_name: YYYY-MM-DD_HH-MM-SS
'''
# Jarvis:
annotations/dataset_name/
                ├── trial1/
                │   ├── camA/
                │   │    ├── frame1.jpg
                │   │    ├── frame2.jpg
                │   │    └── annotations.csv
                │   └── camB/
                │        ├── frame1.jpg
                │        ├── frame2.jpg
                │        └── annotations.csv
                └── trial2/
                    ├── camA/
                    │    ├── frame1.jpg
                    │    ├── frame2.jpg
                    │    └── annotations.csv
                    └── camB/
                         ├── frame1.jpg
                         ├── frame2.jpg
                         └── annotations.csv

# Lightning Pose:
project_name/
    ├──CollectedData.csv
    └──labeled-data/
                ├── trial1/
                │   ├── camA/
                │   │    ├── frame1.jpg
                │   │    └── frame2.jpg
                │   └── camB/
                │        ├── frame1.jpg
                │        └── frame2.jpg
                └── trial2/
                    ├── camA/
                    │    ├── frame1.jpg
                    │    └── frame2.jpg
                    └── camB/
                         ├── frame1.jpg
                         └── frame2.jpg
'''

In [ ]:
jarvis_dir = r'/home/yiting/Documents/Jarvis_projects/Datasets/annotations/6cam_dataset_231216' # path to the Jarivs labeled dataset 
lp_dir = r'/home/yiting/Documents/LP_projects/LP_240726' # path to the lp project

# Convert Jarvis labeled data to Lightning Pose labeled data
J2LP_sigview(jarvis_dir, lp_dir)

# Copy labeled frames to the LP project folder
get_labeledframes(jarvis_dir, lp_dir)

# Check if frame paths are correct
csv_file = os.path.join(lp_dir, "CollectedData.csv")
df_all = pd.read_csv(csv_file, header = [0,1,2], index_col=0)
for im in df_all.index:
    assert os.path.exists(os.path.join(lp_dir, im))

In [ ]:
# Multi-view project
# session_name: YYYY-MM-DD
# trial_name: YYYY-MM-DD_HH-MM-SS
'''
# Jarvis:
annotations/dataset_name/
                ├── trial1/
                │   ├── camA/
                │   │    ├── frame1.jpg
                │   │    ├── frame2.jpg
                │   │    └── annotations.csv
                │   └── camB/
                │        ├── frame1.jpg
                │        ├── frame2.jpg
                │        └── annotations.csv
                └── trial2/
                    ├── camA/
                    │    ├── frame1.jpg
                    │    ├── frame2.jpg
                    │    └── annotations.csv
                    └── camB/
                         ├── frame1.jpg
                         ├── frame2.jpg
                         └── annotations.csv

# Lightning Pose:
project_name/
    ├──camA.csv
    ├──camB.csv
    └──labeled-data/
                ├── trial1/
                │   ├── camA/
                │   │    ├── frame1.jpg
                │   │    └── frame2.jpg
                │   └── camB/
                │        ├── frame1.jpg
                │        └── frame2.jpg
                └── trial2/
                    ├── camA/
                    │    ├── frame1.jpg
                    │    └── frame2.jpg
                    └── camB/
                         ├── frame1.jpg
                         └── frame2.jpg
'''

In [ ]:
jarvis_dir = r'/home/yiting/Documents/Jarvis_projects/Datasets/annotations/6cam_dataset_231216' # path to the Jarivs labeled dataset 
lp_dir = r'/home/yiting/Documents/LP_projects/LP_240725' # path to the lp project

# Convert Jarvis labeled data to Lightning Pose labeled data
J2LP_mulview(jarvis_dir, lp_dir)
# Copy labeled frames to the LP project folder
get_labeledframes(jarvis_dir, lp_dir)

# Check if frame paths are correct
camera_csvs = [filename for filename in os.listdir(lp_dir) if filename.endswith('.csv')]
for c in camera_csvs:
    csv_file = os.path.join(lp_dir, c)
    df_all = pd.read_csv(csv_file, header = [0,1,2], index_col=0)
    for im in df_all.index:
        assert os.path.exists(os.path.join(lp_dir, im))

## Format, rename, organize videos

In [ ]:
# Format, rename, organize videos

# session_name: YYYY-MM-DD
# trial_name: YYYY-MM-DD_HH-MM-SS
'''
Video Source:
videos/session1/
          ├── trial1/
          │   └── camA.mp4
          │   └── camB.mp4
          └── trial2/
              └── camA.mp4
              └── camB.mp4
Lightning Pose:
project_name/videos/
               ├── session1Ttrial1_camA.mp4
               ├── session1Ttrial1_camB.mp4
               ├── session1Ttrial2_camA.mp4
               └── session1Ttrial2_camB.mp4
'''


In [ ]:
jarvis_dir = r'/home/yiting/Documents/Jarvis_projects/Datasets/annotations/6cam_dataset_231216' # path to the Jarivs labeled dataset 
lp_dir = r'/home/yiting/Documents/LP_projects/LP_240726' # path to the lp project
src_vid_dir = r'/home/yiting/Documents/Data/Videos'
get_videos(jarvis_dir, lp_dir, src_vid_dir)


## Get context frames

In [ ]:
# Get context frames 
'''
project_name/labeled-data/
                └── trial1/
                    └── camA/
                         └── frame10.jpg

# context_range = [-2,2]:

project_name/labeled-data/
                └── trial1/
                    └── camA/
                        ├── frame08.jpg
                        ├── frame09.jpg
                        ├── frame10.jpg
                        ├── frame11.jpg
                        └── frame12.jpg
'''

In [ ]:
# Multi-view project
lp_dir = r'/home/yiting/Documents/LP_projects/LP_240726' # path to the lp project
context_range = [-2, 2]
get_contextframes(lp_dir, context_range)


In [ ]:
## Format, rename, organize videos for inference
src_vid_dir = r'/home/yiting/Documents/LP_projects/New_videos'
dst_vid_dir = r'/home/yiting/Documents/LP_projects/LP_240719/new_videos'
trials = os.listdir(src_vid_dir)
for t in trials:
    cameras = os.listdir(os.path.join(src_vid_dir, t))
    trialname_parts = t.split('_')
    for c in cameras:
        new_vid_name = trialname_parts[0] + 'T' + trialname_parts[1] + '_' + c
        # Check video format and reencode video (if needed)
        if check_codec_format(os.path.join(src_vid_dir, t, c)):
            shutil.copy(os.path.join(src_vid_dir, t, c), os.path.join(dst_vid_dir, new_vid_name))
        else:
            reencode_video(os.path.join(src_vid_dir, t, c), os.path.join(dst_vid_dir, new_vid_name))

In [ ]:
## 